In [65]:
import pandas as pd
import numpy as np
from RBFNN import RBFNN
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import glob

In [66]:
# 1. Ambil semua file CSV di folder saat ini yang sesuai pola
csv_files = glob.glob("*.csv")

# 2. Gabungkan semua file
df_list = [pd.read_csv(f) for f in csv_files]
df_all = pd.concat(df_list, ignore_index=True)
# print(df_all)
# Filter hanya baris dengan format filename yang benar (misal: 0_1.png)
# df_all = df_all[df_all['filename'].str.contains(r'^\d+_\d+\.png$', regex=True)]

# # Ekstrak label dari filename
# df_all['label'] = df_all['filename'].str.extract(r'^(\d+)_')[0].astype(int)
# df_all['label'] = df_all['filename'].str.split('_').str[0].astype(int)
df_all['label_str'] = df_all['filename'].str.extract(r'^([^_]+)')[0]

# files = ['Kelas 1 40.csv', 'Kelas 2 40.csv', 'Kelas 3 40.csv', 'Kelas 4 40.csv']
# df_all = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)



In [67]:
df_all['label_cat'] = pd.Categorical(df_all['label_str']).codes
le = LabelEncoder()
df_all['label_enc'] = le.fit_transform(df_all['label_str'])

In [68]:
# X = df_all.drop(columns=['filename', 'label']).values.astype(float)
# y = df_all['label'].values

X = df_all.drop(columns=['filename','label_str','label_cat','label_enc'])
y = df_all['label_enc'].values


In [69]:
# kita akan kumpulkan potongan per label:
X_train_parts, X_test_parts = [], []
y_train_parts, y_test_parts = [], []

# ambil kembali DataFrame agar mudah groupby:
df_all = pd.DataFrame(
    np.hstack([X, y.reshape(-1,1)]),
    columns=[f'pixel_{i}' for i in range(X.shape[1])] + ['label']
)

for lab, group in df_all.groupby('label'):
    # pisah fitur dan target utk setiap grup
    Xg = group.drop('label', axis=1).values.astype(float)
    yg = group['label'].values.astype(int)
    
    # split per grup
    Xg_train, Xg_test, yg_train, yg_test = train_test_split(
        Xg, yg,
        test_size=0.2,
        random_state=42,
        shuffle=True
    )
    
    X_train_parts.append(Xg_train)
    X_test_parts.append(Xg_test)
    y_train_parts.append(yg_train)
    y_test_parts.append(yg_test)

# gabungkan kembali semua grup
X_train = np.vstack(X_train_parts)
X_test  = np.vstack(X_test_parts)
y_train = np.hstack(y_train_parts)
y_test  = np.hstack(y_test_parts)


In [70]:
print("Train distribusi per label:", np.bincount(y_train))
print("Test  distribusi per label:", np.bincount(y_test))

Train distribusi per label: [79 79 79 79 79 79 79 79 79 79 78 78 78 78 78]
Test  distribusi per label: [20 20 20 20 20 20 20 20 20 20 20 20 20 20 20]


In [71]:
# 5. Latih model
model = RBFNN(n_centers=10)
model.fit(X_train, y_train)

# 6. Prediksi & evaluasi
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.056666666666666664
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        20
           1       0.00      0.00      0.00        20
           2       0.00      0.00      0.00        20
           3       0.00      0.00      0.00        20
           4       0.00      0.00      0.00        20
           5       0.03      0.05      0.04        20
           6       0.06      0.20      0.09        20
           7       0.10      0.40      0.16        20
           8       0.01      0.05      0.02        20
           9       0.07      0.10      0.08        20
          10       0.00      0.00      0.00        20
          11       0.00      0.00      0.00        20
          12       1.00      0.05      0.10        20
          13       0.00      0.00      0.00        20
          14       0.00      0.00      0.00        20

    accuracy                           0.06       300
   macro avg       0.08      0.06      0.03      

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
